<a href="https://colab.research.google.com/github/mooochiiii/exercise/blob/main/%C4%91%E1%BB%93_%C3%A1n_th%E1%BB%91ng_k%C3%AA_suy_di%E1%BB%85n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Sử dụng tập tin Diet_R.csv để thực hiện phân tích thống kê suy diễn - ANOVA một chiều. 

In [ ]:
# Các thư viện cần thiết 

import pandas as pd
import xlrd
import numpy as np
import scipy.stats as st
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/notebook/Diet_R.csv")

In [ ]:
# xuất ra 5 dòng dữ liệu đầu 

df.head()

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
0,25,,41,171,60,2,60.0
1,26,,32,174,103,2,103.0
2,1,0,22,159,58,1,54.2
3,2,0,46,192,60,1,54.0
4,3,0,55,170,64,1,63.3


**Câu hỏi 1 được đặt ra: Chế độ ăn kiêng nào tốt nhất để giảm cân?**

- Dựa vào bảng số liệu nhận thấy có 3 chế độ. Vì vậy cần chia dữ liệu thành các nhóm có liên quan đến 3 chế độ này.

- Từ các nhóm trên chúng ta sẽ tiến hành phân tích thông kê suy diễn ANOVA một chiều 



In [ ]:
diet1 = df[df["Diet"] == 1]
diet2 = df[df["Diet"] == 2]
diet3 = df[df["Diet"] == 3]

In [ ]:
weight_loss1 = diet1["pre.weight"] - diet1["weight6weeks"]
weight_loss2 = diet2["pre.weight"] - diet2["weight6weeks"]
weight_loss3 = diet3["pre.weight"] - diet3["weight6weeks"]

In [ ]:
print(weight_loss1)

2     3.8
3     6.0
4     0.7
5     2.9
6     2.8
7     2.0
8     2.0
9     8.5
10    1.9
11    3.1
12    1.5
13    3.0
14    3.6
15    0.9
45   -0.6
46    1.1
47    4.5
48    4.1
49    9.0
50    2.4
51    3.9
52    3.5
53    5.1
54    3.5
dtype: float64


In [ ]:
print(weight_loss2)

0     0.0
1     0.0
16   -2.1
17    2.0
18    1.7
19    4.3
20    7.0
21    0.6
22    2.7
23    3.6
24    3.0
25    2.0
26    4.2
27    4.7
28    3.3
29   -0.5
55    4.2
56    2.4
57    5.8
58    3.5
59    5.3
60    1.7
61    5.4
62    6.1
63    7.9
64   -1.4
65    4.3
dtype: float64


In [ ]:
print(weight_loss3)

30    7.0
31    5.6
32    3.4
33    6.8
34    7.8
35    5.4
36    6.8
37    7.2
38    7.0
39    7.3
40    0.9
41    7.6
42    4.1
43    6.3
44    5.0
66    2.5
67    0.9
68    3.5
69    0.5
70    2.8
71    8.6
72    4.5
73    2.8
74    4.1
75    5.3
76    9.2
77    6.1
dtype: float64


In [ ]:
# Xây dựng DataFrame

df1 = pd.DataFrame({
    "weight loss1": weight_loss1,
    "weight loss2": weight_loss2,
    "weight loss3": weight_loss3
})

In [ ]:
# Kiểm tra lại kết quả 

df1

,weight loss1,weight loss2,weight loss3
0,NaN,0.0,NaN
1,NaN,0.0,NaN
2,3.8,NaN,NaN
3,6.0,NaN,NaN
4,0.7,NaN,NaN
...,...,...,...
73,NaN,NaN,2.8
74,NaN,NaN,4.1
75,NaN,NaN,5.3
76,NaN,NaN,9.2


In [ ]:
# Tiến hành xoá NaN của từng nhóm

list_non_na = [list(df1["weight loss1"].dropna()),
               list(df1["weight loss2"].dropna()),
               list(df1["weight loss3"].dropna())]

**Tiến hành kiểm định các nhóm có lấy từ phân phối chuẩn**

- Xây dựng giả thuyết với mức ý nghĩa 0.05:
 - Giả thuyết null: 

 $H_0$: Các nhóm được lấy từ phân phối chuẩn

 - Giả thuyết thay thế:

 $H_a$: Các nhóm không được lấy từ phân phối chuẩn

In [ ]:
for i in range(3):
  p_value = stats.shapiro(list_non_na[i])[1]
  print(f"P_value của cột thứ {i+1}: {p_value}")
  if p_value < 0.05:
    print(f"Có bằng chứng cho thấy nhóm weight loss {i+1} không tuân theo phân phối chuẩn")
  else:
    print(f"Không có bằng chứng cho thấy nhóm weight loss {i+1} không tuân theo phân phối chuẩn")

P_value của cột thứ 1: 0.07748671621084213
Không có bằng chứng cho thấy nhóm weight loss 1 không tuân theo phân phối chuẩn
P_value của cột thứ 2: 0.961176335811615
Không có bằng chứng cho thấy nhóm weight loss 2 không tuân theo phân phối chuẩn
P_value của cột thứ 3: 0.37203603982925415
Không có bằng chứng cho thấy nhóm weight loss 3 không tuân theo phân phối chuẩn


**Tiến hành kiểm định phương sai của các nhóm có sự khác biệt**

Xây dựng giả thuyết với mức ý nghĩa 0.05:
 - Giả thuyết null:

 $H_0$: Các nhóm có phương sai giống nhau 
 - Giả thuyết thay thế:
 
 $H_a$: Các nhóm có phương sai không giống nhau 

In [ ]:
stats, p_value = st.bartlett(weight_loss1, weight_loss2, weight_loss3)
print("p_value:",p_value)
if p_value < 0.05:
  print("Có bằng chứng bác bỏ  giả thuyết H0")
else:
  print("Không có bằng chứng bác bỏ  giả thuyết H0")

p_value: 0.8447392194944549
Không có bằng chứng bác bỏ  giả thuyết H0


**Ta tiến hành phân tích ANOVA 1 chiều với dữ liệu vì các nhóm đều độc lập, được lấy từ phân phối chuẩn và giống về phương sai:**

Phát biểu giả thuyết:
 - Giả thuyết null:

 $H_0$: Không tồn tại sự khác biệt giữa 3 chế độ ăn kiêng 
 - Giả thuyết thay thế:
 
 $H_a$: Tồn tại sự khác biệt giữa 3 chế độ ăn kiêng    

In [ ]:
stats, p_value = st.f_oneway(weight_loss1, weight_loss2, weight_loss3)
print("p_value:",p_value)
if p_value < 0.05:
  print("Có bằng chứng bác bỏ giả thuyết H0")
else:
  print("Không có bằng chứng bác bỏ giả thuyết H0")

p_value: 0.0032290142385893524
Có bằng chứng bác bỏ giả thuyết H0


**Tiến hành kiểm định POST-HOC để tìm ra chính xác cặp giá trị nào đang có sự khác biệt**

- Sử dụng thư viện StatsModels 
- Sử dụng phương pháp Tuker trong thư viện StatsModels để thực hiện kiểm định

In [ ]:
# import thư viện cần thiết 

import statsmodels.stats.multicomp as mc

In [ ]:
# tạo DataFrame

df_2 = pd.DataFrame({'diet': list(np.concatenate([weight_loss1, weight_loss2, weight_loss3]).flat),
                   'group': np.repeat(['1', '2', '3'], repeats=len(list(np.concatenate([weight_loss1, weight_loss2, weight_loss3]).flat))/3)}) 

In [ ]:
# Tiến hành kiểm định Tuker

comp = mc.MultiComparison(df_2['diet'], df_2['group'])
post_hoc_res = comp.tukeyhsd()
post_hoc_res.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
1,2,0.3654,0.8332,-1.2415,1.9723,False
1,3,2.0308,0.0095,0.4238,3.6377,True
2,3,1.6654,0.0405,0.0585,3.2723,True


- Sau khi thực hiện kiểm định ANOVA ta kết luận được 3 chế độ ăn kiêng có sự khác biệt. 

- Bây giờ, ta chỉ cần dự trên sự biến đổi của trọng lượng trung bình của từng chế độ ăn kiêng để kết luận chế độ nào hiệu quả nhất. 

In [ ]:
mean= [weight_loss1.mean(), weight_loss2.mean(), weight_loss3.mean()]
mode = ['1', '2', '3']
table = list(zip(mode, mean))
table = pd.DataFrame(table)
table.columns = ['Diet','Mean']
table


,Diet,Mean
0,1,3.300000
1,2,3.025926
2,3,5.148148


Từ bảng trên ta suy ra được rằng chế độ ăn kiêng thứ 3 là tốt nhất để giảm cân.

**Câu hỏi 2 được đặt ra: khả năng giảm cân có phụ thuộc vào giới tính hay không?**

- Dựa vào bảng số liệu nhận thấy có 2 giới tính. Vì vậy cần chia dữ liệu thành các nhóm dựa trên 2 giới tính.

- Từ các nhóm trên chúng ta sẽ tiến hành phân tích thông kê suy diễn ANOVA một chiều 

In [ ]:
# thêm cột weight_loss vào DataFrame ban đầu

weight_loss = df["pre.weight"] - df["weight6weeks"]
df["weight loss"] = weight_loss.values
df

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks,weight loss
0,25,,41,171,60,2,60.0,0.0
1,26,,32,174,103,2,103.0,0.0
2,1,0,22,159,58,1,54.2,3.8
3,2,0,46,192,60,1,54.0,6.0
4,3,0,55,170,64,1,63.3,0.7
...,...,...,...,...,...,...,...,...
73,74,1,35,183,83,3,80.2,2.8
74,75,1,49,177,84,3,79.9,4.1
75,76,1,28,164,85,3,79.7,5.3
76,77,1,40,167,87,3,77.8,9.2


In [ ]:
# tạo dataframe chứa 2 cột nam và nữ dự trên số cân nặng

nam = df["weight loss"][df["gender"]== "0"]
nu = df["weight loss"][df["gender"]== "1"]

df2 = pd.DataFrame({"Nam": nam, "Nữ": nu})
df2

,Nam,Nữ
2,3.8,NaN
3,6.0,NaN
4,0.7,NaN
5,2.9,NaN
6,2.8,NaN
...,...,...
73,NaN,2.8
74,NaN,4.1
75,NaN,5.3
76,NaN,9.2


In [ ]:
# Tiến hành xoá NaN của từng nhóm
list_1 = [list(df2["Nam"].dropna()),
          list(df2["Nữ"].dropna())]

**Tiến hành kiểm định các nhóm có lấy từ phân phối chuẩn**

- Xây dựng giả thuyết với mức ý nghĩa 0.05:
 - Giả thuyết null: 

 $H_0$: Các nhóm được lấy từ phân phối chuẩn

 - Giả thuyết thay thế:

 $H_a$: Các nhóm không được lấy từ phân phối chuẩn

In [ ]:
stats, p_value = st.shapiro(df2['Nam'])
print("p_value:", p_value)
if p_value < 0.05:
  print("Có bằng chứng cho thấy nhóm nam không tuân theo phân phối chuẩn" )
else:
  print("Không có bằng chứng cho thấy nhóm nam không tuân theo phân phối chuẩn ")

p_value: 1.0
Không có bằng chứng cho thấy nhóm nam không tuân theo phân phối chuẩn 


In [ ]:
stats, p_value = st.shapiro(df2["Nữ"])
print("p_value:", p_value)
if p_value < 0.05:
  print("Có bằng chứng cho thấy nhóm nữ không tuân theo phân phối chuẩn" )
else:
  print("Không có bằng chứng cho thấy nhóm nữ không tuân theo phân phối chuẩn ")

p_value: 1.0
Không có bằng chứng cho thấy nhóm nữ không tuân theo phân phối chuẩn 


**Tiến hành kiểm định phương sai của các nhóm có sự khác biệt**

Xây dựng giả thuyết với mức ý nghĩa 0.05:
 - Giả thuyết null:

 $H_0$: Các nhóm có phương sai giống nhau 
 - Giả thuyết thay thế:
 
 $H_a$: Các nhóm có phương sai không giống nhau 

In [ ]:
stats, p_value = st.bartlett(nam, nu)
print("p_value:",p_value)
if p_value < 0.05:
  print("Có bằng chứng bác bỏ  giả thuyết H0")
else:
  print("Không có bằng chứng bác bỏ  giả thuyết H0")

p_value: 0.9735995151820288
Không có bằng chứng bác bỏ  giả thuyết H0


**Ta tiến hành phân tích ANOVA 1 chiều với dữ liệu vì các nhóm đều độc lập, được lấy từ phân phối chuẩn và giống về phương sai:**

Phát biểu giả thuyết:
 - Giả thuyết null:

 $H_0$: Khả năng giảm cân không phụ thuộc vào giới tính 
 - Giả thuyết thay thế:
 
 $H_a$: Khả năng giảm cân có phụ thuộc vào giới tính 

In [ ]:
stats, p_value = st.f_oneway(nam, nu)
print("p_value:",p_value)
if p_value < 0.05:
  print("Có bằng chứng bác bỏ giả thuyết H0")
else:
  print("Không có bằng chứng bác bỏ giả thuyết H0")

p_value: 0.8348273886348012
Không có bằng chứng bác bỏ giả thuyết H0
